In [2]:
from openpyxl import load_workbook
from html.parser import HTMLParser
import requests,base64,os
from xml.etree import ElementTree  as ET
print('aaa')

aaa


In [92]:
def get_excel_info(infile,hospSampleID,flag = '5'):
    result_dct_lst = []
    all_lis_item_code =  ['32378', '32379', '32380', '32381', '32383']
#     patient_info = get_patient_info(hospSampleID)
    dct = {}
#     dct = get_LisRequest_data(patient_info)
    ex  = load_workbook(infile)
    tb = ex['报告']
    dct['lis_Barcode'] = tb.cell(row=4,column=3).value
    # 采样日期：	
    dct['blood_time'] = tb.cell(row=6,column=3).value
    dct['ext_receive_time'],dct['ext_check_time'],dct['ext_report_time'] =  tb.cell(row=7,column=3).value, \
                                                                            tb.cell(row=7,column=7).value, \
                                                                            tb.cell(row=7,column=10).value
   # sheet里面从B10到H16,即检测结果的内容：
    result_info = ''
    i  = 0
    all_lis_item_code =  ['32378', '32379', '32380', '32383', '32381','32382']
    all_lis_item_name =  ['睾酮', '雄烯二酮', '硫酸脱氢表雄酮', '双氢睾酮', '脱氢表雄酮','17α-羟孕酮']
    all_lis_combine_code = ['32025']*5 + ['32024']
    for row in range(10,16):
        result_dct = {}
        i += 1
        if flag == '5' and i ==6:
            continue 
        if flag == '17' and i  in (1,2,3,4,5):
            continue 
        print(i)
        result_dct['ext_item_code'] = all_lis_item_code[i-1]
        result_dct['lis_item_name'] = all_lis_item_name[i-1]

        # ext_item_name = tb.cell(row=row,column=2).value
        result_dct['lis_item_code'] = all_lis_item_code[i-1]  
        result_dct['result'] = tb.cell(row=row,column=7).value   
        result_dct['lis_combine_code'] = all_lis_combine_code[i-1]

        print('lis_item_nam--------')
        print(all_lis_item_name[i-1],all_lis_item_code[i-1] ,result_dct['result'] )            
        result_dct['ext_item_code'] = all_lis_item_code[i-1]
        # ext_item_name = tb.cell(row=row,column=2).value
#         result_dct['lis_item_code'] = tb.cell(row=row,column=13).value     
        # lis_item_name =  tb.cell(row=row,column=2).value
        result_dct['result'] = tb.cell(row=row,column=7).value    
        result_dct_lst.append(result_dct)
#         print(tb.cell(row=row,column=7).value    )
#         result_dct['ext_combine_code'] = dct['lis_combine_code']   
        
        # result_unit = tb.cell(row=10,column=11).value  
    
        # param = [ext_item_code,result,result_unit,lis_item_code,lis_combine_code,ext_combine_code]
#         result_info += set_result_info(result_dct)
    result_info = result_info + '<report_pic><pic_content>'  
    # print(lis_Barcode, blood_time, ext_receive_time, ext_check_time, ext_report_time)

    return result_dct_lst, result_info ,  dct

In [93]:
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈嘉绮-激素检测报告-2022-05-25.xlsx'

In [94]:
result_dct_lst,result_info ,  dct = get_excel_info(f,'3243146536',flag = '5')

1
lis_item_nam--------
睾酮 32378 0.456
2
lis_item_nam--------
雄烯二酮 32379 2.826
3
lis_item_nam--------
硫酸脱氢表雄酮 32380 1596.602
4
lis_item_nam--------
双氢睾酮 32383 0.101
5
lis_item_nam--------
脱氢表雄酮 32381 6.255


In [96]:
result_dct_lst


[{'ext_item_code': '32378',
  'lis_item_name': '睾酮',
  'lis_item_code': '32378',
  'result': 0.456,
  'lis_combine_code': '32025'},
 {'ext_item_code': '32379',
  'lis_item_name': '雄烯二酮',
  'lis_item_code': '32379',
  'result': 2.826,
  'lis_combine_code': '32025'},
 {'ext_item_code': '32380',
  'lis_item_name': '硫酸脱氢表雄酮',
  'lis_item_code': '32380',
  'result': 1596.602,
  'lis_combine_code': '32025'},
 {'ext_item_code': '32383',
  'lis_item_name': '双氢睾酮',
  'lis_item_code': '32383',
  'result': 0.101,
  'lis_combine_code': '32025'},
 {'ext_item_code': '32381',
  'lis_item_name': '脱氢表雄酮',
  'lis_item_code': '32381',
  'result': 6.255,
  'lis_combine_code': '32025'}]

In [97]:
ex  = load_workbook(infile)
tb = ex['报告']
dct['lis_Barcode'] = tb.cell(row=4,column=3).value
# 采样日期：	
dct['blood_time'] = tb.cell(row=6,column=3).value

In [90]:
tb.cell(row=10,column=7).value

0.456

In [91]:
for row in range(10,16):
    print(row)

10
11
12
13
14
15


In [60]:
    all_lis_item_code =  ['32378', '32379', '32380', '32383', '32381']
    all_lis_item_name =  ['睾酮', '雄烯二酮', '硫酸脱氢表雄酮', '双氢睾酮', '脱氢表雄酮','17α-羟孕酮']

In [62]:
all_lis_item_name[1]

'雄烯二酮'

In [98]:
s = '3243146536+3343146537'

In [100]:
s.split('+')[-1]

'3343146537'